#### 1. взять любой набор данных для бинарной классификации (можно скачать один с https://archive.ics.uci.edu/ml/datasets.php)
#### 2. сделать feature engineering
#### 3. обучить любой классификатор (какой вам нравится)
#### 4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
#### 5. применить random negative sampling для построения классификатора в новых условиях
#### 6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
#### 7. поэкспериментировать с долей P на шаге 6 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(
    'D:\Study\Machine Learning in Business\Manual\Lesson6/adult.data', sep=',', index_col=False,
    names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship',
           'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])

df.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [3]:
# Проврка пропусков
pd.DataFrame([df.isna().sum(), df.isnull().sum()], index={'na', 'null'})

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
na,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
# Изменение целевой переменной
df['income'] = df['income'].map({' <=50K': 0, ' >50K': 1})

In [5]:
from sklearn.model_selection import train_test_split

X = pd.get_dummies(df.iloc[:, :-1])
y = df['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [8]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [6]:
import xgboost as xgb

model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

In [9]:
def evaluate_results(y_test, y_predict):
    """ Проверка качества модели с помощью DS-метрик"""
    from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score

    f1 = f1_score(y_test, y_predict)
    roc = roc_auc_score(y_test, y_predict)
    precision = precision_score(y_test, y_predict, average='binary')
    recall = recall_score(y_test, y_predict, average='binary')
    return f1, precision, recall, roc

In [10]:
evaluate_results(y_test, y_predict)

(0.7090464547677262,
 0.7801691006917756,
 0.6498079385403329,
 0.7960209153416673)

Разделим набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

Представим, что нам неизвестны негативы и часть позитивов

In [11]:
mod_data = pd.get_dummies(df.copy())
mod_data['income'].value_counts()

0    24720
1     7841
Name: income, dtype: int64

In [12]:
# получим индексы положительных выборок
pos_ind = np.where(mod_data.loc[:, 'income'].values == 1)[0]

# перемешаем их
np.random.shuffle(pos_ind)

# оставим отмеченными только 25% положительных результатов
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 1961/7841 as positives and unlabeling the rest


In [13]:
# Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
mod_data['class_test'] = -1
mod_data.loc[pos_sample, 'class_test'] = 1
print('target variable:\n', mod_data.iloc[:, -1].value_counts())

target variable:
 -1    30600
 1     1961
Name: class_test, dtype: int64


In [14]:
x_data = mod_data.drop(['income', 'class_test'], axis=1).values  # just the X
y_labeled = mod_data.loc[:, 'class_test'].values  # new class (just the P & U)
y_positive = mod_data.loc[:, 'income'].values  # original class

Применим random negative sampling для построения классификатора в новых условиях

In [15]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test'] == -1][:len(mod_data[mod_data['class_test'] == 1])]
sample_test = mod_data[mod_data['class_test'] == -1][len(mod_data[mod_data['class_test'] == 1]):]
pos_sample = mod_data[mod_data['class_test'] == 1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(1961, 110) (1961, 110)


In [16]:
model_rns = xgb.XGBClassifier()
model.fit(sample_train.drop(['income', 'class_test'], axis=1).values,
          sample_train.loc[:, 'income'].values)
y_predict_rns = model.predict(sample_test.drop(['income', 'class_test'], axis=1).values)

evaluate_results(sample_test.loc[:, 'income'].values, y_predict_rns)

(0.6074129168216189, 0.460742830277386, 0.8910711038370612, 0.821615932212394)

Сравним качество с решением из пункта 4 (отчет - таблица метрик)

In [18]:
pd.DataFrame([
    evaluate_results(y_test, y_predict),
    evaluate_results(sample_test.loc[:, 'income'].values, y_predict_rns)
], columns={'f1', 'precision', 'recall', 'roc'}, index={'Simple xgboost', 'Random negative sampling'}).round(2)

,recall,roc,precision,f1
Simple xgboost,0.71,0.78,0.65,0.80
Random negative sampling,0.61,0.46,0.89,0.82
